In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle

In [3]:
data=pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
data=data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

In [5]:
## Encode categorical variables
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

In [6]:
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [7]:
## One Hot Encoding for 'Geography'
onehot_encoder_geo = OneHotEncoder(handle_unknown='ignore')
geo_encoded = onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
data = pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [8]:
## Split the data into features and target variable
X = data.drop('EstimatedSalary', axis=1)
y = data['EstimatedSalary']

In [9]:
## Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
## Scale these features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [11]:
## Save the encoders and scalar
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo, file)

with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

## ANN Regression Problem statement

In [12]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [13]:
## Build our ANN Model
model=Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)), ## HL! connected with input layer
    Dense(32, activation='relu'), ## HL2
    Dense(1) ## Output layer for regression
])

## Compile the model
model.compile(optimizer='adam', loss='mean_absolute_error', metrics=['mae'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [16]:
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
import datetime

## Setup the tensorboard
log_dir="regressionlogs/fit/" +datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback=TensorBoard(log_dir=log_dir, histogram_freq=1)

In [17]:
## Setup early stopping
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [18]:
## Train the model
history = model.fit(
    X_train, y_train,
    validation_data = (X_test, y_test),
    epochs=100,
    callbacks = [early_stopping_callback, tensorboard_callback]
)

Epoch 1/100
250/250 [==============================] - 0s 1ms/step - loss: 100360.3125 - mae: 100360.3125 - val_loss: 98449.9531 - val_mae: 98449.9531
Epoch 2/100
250/250 [==============================] - 0s 667us/step - loss: 99379.6797 - mae: 99379.6797 - val_loss: 96470.1797 - val_mae: 96470.1797
Epoch 3/100
250/250 [==============================] - 0s 658us/step - loss: 95951.7422 - mae: 95951.7422 - val_loss: 91480.9609 - val_mae: 91480.9609
Epoch 4/100
250/250 [==============================] - 0s 656us/step - loss: 89386.9922 - mae: 89386.9922 - val_loss: 83535.6250 - val_mae: 83535.6250
Epoch 5/100
250/250 [==============================] - 0s 660us/step - loss: 80200.2109 - mae: 80200.2109 - val_loss: 73797.6719 - val_mae: 73797.6719
Epoch 6/100
250/250 [==============================] - 0s 661us/step - loss: 70129.5781 - mae: 70129.5781 - val_loss: 64537.4844 - val_mae: 64537.4844
Epoch 7/100
250/250 [==============================] - 0s 665us/step - loss: 61366.6445 - mae:

In [19]:
%load_ext tensorboard

In [21]:
%tensorboard --logdir regressionlogs/fit

Reusing TensorBoard on port 6009 (pid 99772), started 0:00:31 ago. (Use '!kill 99772' to kill it.)

In [22]:
## Evaluatae the model on the test data
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}, Test MAE: {test_mae}")

63/63 [==============================] - 0s 371us/step - loss: 50304.6094 - mae: 50304.6094
Test Loss: 50304.609375, Test MAE: 50304.609375


In [23]:
model.save('regression_model.h5')

/Users/bhaveshduttmyla/iCloud Drive (Archive)/Documents/Python/ANN Classification/venv/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
